<a href="https://colab.research.google.com/github/KrishnaRupanagunta/Fun-Python-Stuff/blob/main/Family%20Tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

data = [['Rupan1','M','M','1942','','Origin'],
 ['Rup2','M','F','1943','Rupan1','Spouse'],
 ['Rup3','M','M','1965','Rupan1','Child'],
 ['Rup4','M','F','1967','Rup3','Spouse'],
 ['Rup9','M','F','1970','Rupan1','Child'],
 ['Rup5','S','M','1968','Rupan1','Child'],
 ['Rup6','M','F','1967','Rup3','Child'],
 ['Rup7','M','F','1968','Rup6','Spouse'],
 ['Rup10','M','F','1971','Rup9','Spouse']]

rawdf = pd.read_csv('sample_ancestry.csv', dtype= str, keep_default_na=False)
# Create the pandas DataFrame
df = pd.DataFrame(rawdf, columns=['Name1', 'MStatus','Gender','YOB','Name2','Relation'])

#add columns
df['Name1ID'] = df['Name1'].str[:3] + df['YOB']
df['CoupleID'] = ''
df['ParentCoupleID'] = ''
df['Name2ID'] =''

df2=df.copy()

for index, row in df.iterrows():
  result = df2.loc[df2['Name1']== row['Name2'], 'Name1ID']
  if not result.empty:
    value_ID = result.values[0]
    df.loc[index,'Name2ID'] = value_ID

for index, row in df.iterrows():
  if row['Relation'] == 'Spouse':
      valueID= row['Name1ID'] + "." + row['Name2ID']
      row['CoupleID'] = valueID


df2=df.copy()
for index, row in df.iterrows():
  if row['Relation'] == 'Child':
    result = df2.loc[(df2['Name2ID']== row['Name2ID']), 'CoupleID']
    if not result.empty:
      value_ID = result.values[0]
      df.loc[index,'ParentCoupleID'] = value_ID

df2=df.copy()
for index, row in df.iterrows():
  if row['Relation'] == 'Spouse':
    result = df2.loc[(df2['Name1ID']== row['Name2ID']), 'ParentCoupleID']
    if not result.empty:
      value_ID = result.values[0]
      df.loc[index,'ParentCoupleID'] = value_ID
print(df)
graphdf = pd.DataFrame(columns=['CoupleID','ParentCoupleID','CoupleName','MStatus'])

#graph of couples (married)
for index, row in df.iterrows():
  if (row['MStatus'] == 'M') & (row['Relation'] == 'Spouse'):
    CID= row['CoupleID']
    PCID = row['ParentCoupleID']
    CNAME = row['Name2'] + "*," + row['Name1']
    graphdf.loc[len(graphdf.index)] = [CID,PCID,CNAME,'M']

#append singles
for index, row in df.iterrows():
  if (row['MStatus'] == 'S') & (row['Relation'] == 'Child'):
    CID= row['Name1ID']
    PCID = row['ParentCoupleID']
    CNAME = row['Name1'] + "*" + "  "
    graphdf.loc[len(graphdf.index)] = [CID,PCID,CNAME,'S']




In [34]:
from graphviz import Digraph
f = Digraph('neato', format='pdf', encoding='utf8', filename='Rupanagunta', node_attr={'color': 'lightblue', 'style': 'filled'})
f.attr('node', shape='box')
for index, row in graphdf.iterrows():
    f.node(row['CoupleID'],
           label=
             row['CoupleName'],
            _attributes={'color':'lightblue' if row['MStatus']=='M' else 'lightgray'})
for index, row in graphdf.iterrows():
    f.edge(str(row["ParentCoupleID"]), str(row["CoupleID"]), label='')
f.view()

'Rupanagunta.pdf'